<a href="https://colab.research.google.com/github/Uriel1201/HelloSQL/blob/main/04_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. Time difference between latest actions.
From the following table of user actions, write a query to return for each user the time elapsed between the last action and the second-to-last action, in ascending order by user ID.

In [ ]:
import pandas as pd
import numpy as  np

data = {'user_id' : [1,1,2,2,3,3,4,1],
        'actions'    : ['start','cancel','start',
                    'publish','start','cancel',
                    'start','publish'],
        'action_date'   : ['12-FEB-20',
                         '13-FEB-20',
                         '11-FEB-20',
                         '14-FEB-20',
                         '15-FEB-20',
                         '15-FEB-20',
                         '18-FEB-20',
                         '19-FEB-20']
        }

users = pd.DataFrame(data)
users

,user_id,actions,action_date
0,1,start,12-FEB-20
1,1,cancel,13-FEB-20
2,2,start,11-FEB-20
3,2,publish,14-FEB-20
4,3,start,15-FEB-20
5,3,cancel,15-FEB-20
6,4,start,18-FEB-20
7,1,publish,19-FEB-20


In [ ]:
users['action_date'] = pd.to_datetime(users['action_date'])

<ipython-input-2-9319b3a855ed>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  users['action_date'] = pd.to_datetime(users['action_date'])


In [ ]:
users

,user_id,actions,action_date
0,1,start,2020-02-12
1,1,cancel,2020-02-13
2,2,start,2020-02-11
3,2,publish,2020-02-14
4,3,start,2020-02-15
5,3,cancel,2020-02-15
6,4,start,2020-02-18
7,1,publish,2020-02-19


In [ ]:
def row_num(x):
    x_s = pd.Series(x)
    row_n = range(len(x), 0, -1)
    return row_n

In [ ]:
users['date_rank'] = users.groupby('user_id')['action_date'].transform(row_num)

In [ ]:
users

,user_id,actions,action_date,date_rank
0,1,start,2020-02-12,3
1,1,cancel,2020-02-13,2
2,2,start,2020-02-11,2
3,2,publish,2020-02-14,1
4,3,start,2020-02-15,2
5,3,cancel,2020-02-15,1
6,4,start,2020-02-18,1
7,1,publish,2020-02-19,1


In [ ]:
last_act = users[['user_id', 'action_date']][users['date_rank'] == 1]
last_act.rename(columns = {'action_date' : 'last'}, inplace = True)

In [ ]:
last_act

,user_id,last
3,2,2020-02-14
5,3,2020-02-15
6,4,2020-02-18
7,1,2020-02-19


In [ ]:
slast_act = users[['user_id', 'action_date']][users['date_rank'] == 2]
slast_act.rename(columns = {'action_date' : 'penultimate'}, inplace = True)

In [ ]:
slast_act

,user_id,penultimate
1,1,2020-02-13
2,2,2020-02-11
4,3,2020-02-15


In [ ]:
elapsed_days = pd.merge(last_act, slast_act, on = 'user_id', how = 'left').sort_values(by = 'user_id')
elapsed_days.reset_index(drop = True, inplace = True)

In [ ]:
elapsed_days

,user_id,last,penultimate
0,1,2020-02-19,2020-02-13
1,2,2020-02-14,2020-02-11
2,3,2020-02-15,2020-02-15
3,4,2020-02-18,NaT


In [ ]:
elapsed_days['elapsed_time'] = elapsed_days['last'] - elapsed_days['penultimate']

In [ ]:
elapsed_days

,user_id,last,penultimate,elapsed_time
0,1,2020-02-19,2020-02-13,6 days
1,2,2020-02-14,2020-02-11,3 days
2,3,2020-02-15,2020-02-15,0 days
3,4,2020-02-18,NaT,NaT


In [22]:
elapsed_days[['user_id', 'elapsed_time']].sort_values(by = 'user_id')

,user_id,elapsed_time
0,1,6 days
1,2,3 days
2,3,0 days
3,4,NaT
